# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

In [1]:
install.packages("httr")
install.packages("tokenizers")
library(httr)
library(tokenizers)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘SnowballC’




## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [2]:
tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase=TRUE, strip_punct=TRUE)[[1]]
}

#### b) Make a function generate keys for ngrams.

In [3]:
key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse=sep)
}

#### c) Make a function to build an ngram table.

In [4]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))
    tbl <- new.env(parent = emptyenv())
    for (i in seq_len(length(tokens) - n + 1L)) {
        ngram <- tokens[i:(i + n - 2L)]
        next_word <- tokens[i + n - 1L]
        key <- paste(ngram, collapse = sep)
        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }
    tbl
}

#### d) Function to digest the text.

In [5]:
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [6]:
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt,n)
}

#### f) Function that gives random start.

In [7]:
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names=TRUE)
    if (length(keys)==0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed=TRUE)[[1]]
}

#### g) Function to predict the next word.

In [8]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if(!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size=1, prob=as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [9]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl); n <- as.integer(n); force(sep)
    function(start_words = NULL, length = 10L) {
        if ((is.null(start_words)) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep=sep)
        }
        word_sequence <- start_words
        for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep=sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }
        paste(word_sequence, collapse= " ")
    }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [14]:
set.seed(2025)
url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)

print(gen3(start_words=c("the","king"),length=15))
print(gen3(length=15))

[1] "the king has forbidden me to marry another husband am not i shall ride upon"
[1] "song was over the lake and herself into her little daughter’s hand and was about"


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [15]:
set.seed(2025)
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)

print(gen3(start_words=c("the", "king"),length=15))
print(gen3(length=15))

[1] "the king he added to the entire exclusion of the swords were made prisoners the"
[1] "king was campaigning in france denmark germany switzerland and livonia figures 5 and the sword"


#### c) Explain in 1-2 sentences the difference in content generated from each source.

**The content generated from Grimm's Fairy Tales is largely like a fairytale. In contrast, the second source is more historical and has elements of war and conques with the use of weapons, prisoners, countries.**

## Question 3
#### a) What is a language learning model?
#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

**a)A lanuguage model is a type of machine learning model that predicts the probability of a sequence of words to understand and generate human language.**

**b)You can run a language model locally by using Docker or Ollama, which allow you to download the model and run it entirely offline.**

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | The program that interprets and runs the command. |
| **Terminal emulator** | The application window that hosts the shell and displays the text that you type along with the shell's response. |
| **Process** | It is a running program. |
| **Signal** | A message that is set to a process to tell it to do something. Signals are not used here but the mkdir could recieve them. |
| **Standard input** | A stream a process can read from. mkdir does not read from the standard input. |
| **Standard output** | This is where a process writes text output and typically mkdir writes nothing unless there is an error. |
| **Command line argument** | This is extra text that is passed into a program. in mkdir project the project portion is the argument that tells mkdir which directory to make. |
| **The environment** | It is the set of variables avaialbe to the process. The shell uses the environment to find the mkdir program and run it. |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
#### b) Explain what this command is doing, part by part.

**a)The programs are find xargs and grep.**

**b)find . -iname "*.R" is searching the current directory (.), and the subdirectories for files whose names end with (.R) irrespective of the case. The pipe (|) sends the list of matchign filenames from find into the next program. xargs grep read_csv takes the filenames it has recieved from find and then passes them as arguements to grep read_csv which then searches inside each file for read_csv.**

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions.
#### a) Show the response when you run `docker run hello-world`.
#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.
#### c) How do you log in to the RStudio server?

**a)**
Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/


**b)**
input: docker run -d -p 8787:8787 -e PASSWORD=mypassword -v ~/Desktop/rprojects:/home/rstudio/projects rocker/rstudio

output: 2e2044093ea6a6b1cb28df51908a0a5dc1af31985539a143c1ded69b7a7c5b18

**c)**
go to localhost:8787, type studio as the username and the password is whatever we set our password to in part b.
